# Backborn 
YOLO V3는 백본으로 DarkNet-53을 사용한다.

In [2]:
# 필요한 라이브러리
import torch
from torch import nn


In [42]:
# 입력이미지 랜덤생성
input_image=torch.randn(1,3,416,416)
print(input_image.shape)

torch.Size([1, 3, 416, 416])


In [43]:
def DBL(in_c,out_c,kernel_size,stride,padding):
    dbl_block=nn.Sequential(nn.Conv2d(in_c,out_c,kernel_size=kernel_size,padding=padding,stride=stride),
                            nn.BatchNorm2d(out_c),
                            nn.LeakyReLU())
    return dbl_block

class Res_unit(nn.Module):
    def __init__(self,in_c):
        super(Res_unit,self).__init__()
        
        reduce_c=int(in_c/2)
        self.layer1=DBL(in_c,reduce_c,1,1,0)
        self.layer2=DBL(reduce_c,in_c,3,1,1)
    
    def forward(self,x):
        res_connection= x
        out=self.layer1(x)
        out=self.layer2(out)
        out=out+res_connection
        return out
    

In [52]:
Res_unit(3).forward(input_image)

tensor([[[[-3.2327e-01, -4.2690e-01, -1.8316e-01,  ..., -9.2198e-01,
           -8.6366e-01,  2.4112e-01],
          [ 3.0145e-01, -6.2363e-01, -1.0028e+00,  ..., -2.7984e-02,
            8.8971e-01,  7.5431e-01],
          [ 1.6673e+00,  1.5661e+00,  4.7620e-01,  ..., -1.7927e+00,
            1.9801e+00,  1.4639e+00],
          ...,
          [-4.0138e-01, -6.1240e-01,  1.7332e-01,  ...,  1.1001e+00,
            2.6275e+00, -1.7887e-01],
          [-8.2034e-01, -1.2909e+00,  1.1272e+00,  ...,  2.9928e-01,
            1.7577e+00,  8.4310e-02],
          [-1.4758e+00, -2.6708e-04,  1.4700e+00,  ...,  5.4825e-01,
           -1.2433e+00, -2.5719e-01]],

         [[-3.0732e-01, -6.4793e-01,  1.1121e+00,  ...,  1.8404e+00,
           -1.5219e+00, -5.1091e-01],
          [-1.2133e-01,  3.6695e-01,  6.4055e-01,  ...,  2.1386e+00,
           -1.2028e+00, -4.1018e-02],
          [-1.7991e+00,  5.7088e-02,  2.4664e-01,  ...,  2.1433e+00,
            3.2705e+00,  5.9722e-01],
          ...,
     

In [99]:
class Darknet53(nn.Module):
    def __init__(self,block):
        super(Darknet53,self).__init__()
        
        self.conv1=DBL(3,32,3,1,1)
        self.conv2=DBL(32,64,3,2,1)
        
        self.res_block1=self.num_block(block,64,num=1)
        self.conv3=DBL(64,128,3,2,1)
        
        self.res_block2=self.num_block(block,128,2)
        self.conv4=DBL(128,256,3,2,1)
        
        self.res_block3=self.num_block(block,256,8)
        self.conv5=DBL(256,512,3,2,1) # 3*3 conv하면 마진1 > 패딩으로 채워줌
        
        self.res_block4=self.num_block(block,512,8)
        self.conv6=DBL(512,1024,3,2,1)
        
        self.res_block5=self.num_block(block,1024,4)

    def forward(self,x):
        x=self.conv1(x)
        print(x.shape)
        x=self.conv2(x)
        print(x.shape)
        x=self.res_block1(x)
        print(x.shape)
        x=self.conv3(x)
        print(x.shape)
        x=self.res_block2(x)
        print(x.shape)
        x=self.conv4(x)
        print(x.shape)
        x=self.res_block3(x)
        feature3=x
        print(x.shape)
        x=self.conv5(x)
        print(x.shape)
        x=self.res_block4(x)
        feature2=x
        print(x.shape)
        x=self.conv6(x)
        print(x.shape)
        x=self.res_block5(x)
        feature1=x
        print(x.shape)
        
        return feature1,feature2,feature3
    def num_block(self,block,in_c,num):
        layers=[]
        for i in range(num):
            layers.append(Res_unit(in_c))
        return nn.Sequential(*layers)
        

In [62]:
Darknet53(Res_unit).forward(input_image)

torch.Size([1, 32, 416, 416])
torch.Size([1, 64, 208, 208])
torch.Size([1, 64, 208, 208])
torch.Size([1, 128, 104, 104])
torch.Size([1, 128, 104, 104])
torch.Size([1, 256, 52, 52])
torch.Size([1, 256, 52, 52])
torch.Size([1, 512, 26, 26])
torch.Size([1, 512, 26, 26])
torch.Size([1, 1024, 13, 13])
torch.Size([1, 1024, 13, 13])


(tensor([[[[ 5.7497e-01,  7.3823e-01,  1.2205e+00,  ...,  1.5304e+00,
             7.8111e-01,  1.6886e+00],
           [ 1.5839e-01,  8.9254e-01, -4.0790e-02,  ...,  1.0343e+00,
             8.9458e-01,  4.4468e-01],
           [ 5.6743e-01,  1.4764e-01,  1.4916e-02,  ...,  1.0116e+00,
            -8.1921e-03,  5.0945e-02],
           ...,
           [-1.5584e-02, -1.0974e-02,  9.4984e-01,  ...,  1.4502e+00,
             1.5610e-01,  8.5831e-01],
           [ 2.6860e-01,  2.1779e-01,  1.2150e+00,  ...,  1.8456e+00,
            -1.6109e-02,  7.6382e-01],
           [ 1.3128e-01,  3.6126e-01,  1.0463e+00,  ..., -1.6390e-02,
             8.5275e-01,  1.4939e+00]],
 
          [[ 5.9819e-01, -6.4308e-03,  1.4086e+00,  ..., -7.1420e-03,
             2.1642e-02,  2.2628e-01],
           [-2.2193e-03,  1.1318e+00,  1.2095e+00,  ...,  8.9810e-01,
             1.0465e+00,  1.8816e+00],
           [ 5.2624e-01,  1.6689e+00, -1.6892e-02,  ...,  1.4043e+00,
             1.9462e+00, -1.1011e-02],


# Architecture

In [119]:
class Yolo_v3(nn.Module):
    def __init__(self):
        super(Yolo_v3,self).__init__()
        self.darknet53=Darknet53(Res_unit)
        
        self.conv_set1=self.conv_set(1024,512)
        self.conv_final1=self.conv_final(512,255)
        
        self.conv_layer1=DBL(512,256,1,1,0)
        self.upsampling1=nn.Upsample(scale_factor=2,mode='nearest')
        
        self.conv_set2=self.conv_set(768,256) # 왜 나누기 3???????????
        self.conv_final2=self.conv_final(256,255)
        
        self.conv_layer2=DBL(256,128,1,1,0)
        self.upsampling2=nn.Upsample(scale_factor=2,mode='nearest')
        self.conv_set3=self.conv_set(384,128)
        self.conv_final3=self.conv_final(128,255)
        
    def forward(self,x):
        print("darknet53 layer size")
        res5,res4,res3=self.darknet53(x)
        
        print("=========================================")
        print("yolo detect layer size")
        print("res3: ",res3.shape) # res 1이후
        print("res4: ",res4.shape) # res 3이후
        print("res5: ",res5.shape)# 마지막

        # 1번째 feature 뽑기
        out1=self.conv_set1(res5)
        first=self.conv_final1(out1)
        
        # 2번째 feature
        out2=self.conv_layer1(out1)
        out2=self.upsampling1(out2)
        out2=torch.cat((out2,res4),dim=1)
        print("concate1_result:",out2.shape)
        out2=self.conv_set2(out2)
        second=self.conv_final2(out2)

        
        #3번째 feature
        out3=self.conv_layer2(out2)
        out3=self.upsampling2(out3)
        out3=torch.cat((out3,res3),dim=1)
        print("concate2_result:", out3.shape)
        out3=self.conv_set3(out3)
        thrid=self.conv_final3(out3)
        
        # feature 크기출력
        print("first_feature:",first.shape)
        print("second_feature:", second.shape)            
        print("thrid_feature:",thrid.shape)
        
    def conv_set(self,in_c,out_c):
        increase_c=out_c*2
        result=nn.Sequential(DBL(in_c,out_c,1,1,0),
                             DBL(out_c,increase_c,3,1,1),
                             DBL(increase_c,out_c,1,1,0),
                             DBL(out_c,increase_c,3,1,1),
                             DBL(increase_c,out_c,1,1,0))
        return result
    
    def conv_final(self,in_c,out_c):
        result=nn.Sequential(DBL(in_c,in_c*2,3,1,1),
                            nn.Conv2d(in_c*2,out_c,1,1,0))
        return result
    


In [120]:
Yolo_v3().forward(x=input_image)

darknet53 layer size
torch.Size([1, 32, 416, 416])
torch.Size([1, 64, 208, 208])
torch.Size([1, 64, 208, 208])
torch.Size([1, 128, 104, 104])
torch.Size([1, 128, 104, 104])
torch.Size([1, 256, 52, 52])
torch.Size([1, 256, 52, 52])
torch.Size([1, 512, 26, 26])
torch.Size([1, 512, 26, 26])
torch.Size([1, 1024, 13, 13])
torch.Size([1, 1024, 13, 13])
yolo detect layer size
res3:  torch.Size([1, 256, 52, 52])
res4:  torch.Size([1, 512, 26, 26])
res5:  torch.Size([1, 1024, 13, 13])
concate1_result: torch.Size([1, 768, 26, 26])
concate2_result: torch.Size([1, 384, 52, 52])
first_feature: torch.Size([1, 255, 13, 13])
second_feature: torch.Size([1, 255, 26, 26])
thrid_feature: torch.Size([1, 255, 52, 52])
